In [1]:
import numpy as np
import pandas as pd
import functools
from sklearn.cluster import MiniBatchKMeans
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [18]:
train = pd.read_csv('../NYC/train.csv')
test = pd.read_csv('../NYC/test.csv')

## 깨달음
 - merge 함부로 하는 거 아니다, 중복을 항상 살펴라
 - apply 쓰지말고, np 연산을 쓰자
 - clustering MiniBatchKMeans, Birch, KMeans 다양한 알고리즘이 있다.
 
## 반성
 - API 다른 사람이 올려줬는데, 그냥 그대로 쓰지말고 나도 API 가져다 쓸 수 있게 하자

In [2]:
#train, test = addFeature(train.copy(), test.copy())

In [28]:
print(train.shape, test.shape)

(1450352, 26) (625134, 24)


In [ ]:
train.to_csv('train_set2.csv', index = False)
test.to_csv('test_set2.csv', index = False)

In [11]:
def addFeature(train, test):
    #train, test Data
    
    #Time Data
    for df in [train, test]:
        df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
        #df['month'] = df['pickup_datetime'].dt.month
        df['yday'] = df['pickup_datetime'].dt.dayofyear
        df['wday'] = df['pickup_datetime'].dt.dayofweek
        df['hour'] = df['pickup_datetime'].dt.hour
        #df['whour'] = 0
        #df['whour'].loc[(df['hour']<17) & (df['hour']>9)] = 1
        df['store_and_fwd_flag'] = df['store_and_fwd_flag'].map({'N':0, 'Y':1})
    
    train, test = clusterFeature(train, test)
    train, test = distanceFeature(train, test)
    train = cleanDf(train)
    train, test = mergeRouteWeather(train, test)
    #train, test =dummyBuilding(train,test) #Regard of Weather and Route, the potential variables exsits
    
    return train, test

In [10]:
from sklearn.cluster import MiniBatchKMeans
def clusterFeature(train, test):
    coord = np.vstack([train[['pickup_longitude', 'pickup_latitude']].values, train[['dropoff_longitude', 'dropoff_latitude']].values])
    sample_ind = np.random.permutation(len(coord))[:500000]
    kmeans = MiniBatchKMeans(n_clusters = 100, batch_size = 10000).fit(coord[sample_ind])
    train.loc[:,'pickup_cluster'] = kmeans.predict(train[['pickup_longitude', 'pickup_latitude']])
    train.loc[:,'dropoff_cluster'] = kmeans.predict(train[['dropoff_longitude', 'dropoff_latitude']])
    test.loc[:,'pickup_cluster'] = kmeans.predict(test[['pickup_longitude', 'pickup_latitude']])
    test.loc[:,'dropoff_cluster'] = kmeans.predict(test[['dropoff_longitude', 'dropoff_latitude']])
    
    return train, test

1. Direct and Manhatten Distance
https://www.kaggle.com/headsortails/nyc-taxi-eda-update-the-fast-the-curious
2. Distacne JFK/Laguarid_jfk
https://www.kaggle.io/svf/1512124/63b262954485433518d65fb2927eb518/__results__.html#direct-distance-of-the-trip

In [24]:
import functools
def haversine_array(lat1, lng1, lat2, lng2):
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    AVG_EARTH_RADIUS = 6371  # in km
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
    h = 2 * AVG_EARTH_RADIUS * np.arcsin(np.sqrt(d))
    return h

def dummy_manhattan_distance(lat1, lng1, lat2, lng2):
    a = haversine_array(lat1, lng1, lat1, lng2)
    b = haversine_array(lat1, lng1, lat2, lng1)
    return a + b

def distanceFeature(train, test):
    jfk_coord  = [40.639722, -73.778889]
    la_guardia_coord = [40.77725, -73.872611]
    for df in [train, test]:
        df.loc[:, 'jfk_lat'] = jfk_coord[0]
        df.loc[:, 'jfk_lon'] = jfk_coord[1]
        df.loc[:, 'la_lat'] = la_guardia_coord[0]
        df.loc[:, 'la_lon'] = la_guardia_coord[1]
    #Direct Distance
        df.loc[:, 'distance_haversine'] = haversine_array(df['pickup_latitude'].values, df['pickup_longitude'].values, df['dropoff_latitude'].values, df['dropoff_longitude'].values) 
    #Mahattan Distance
        df.loc[:, 'distance_dummy_manhattan'] =  dummy_manhattan_distance(df['pickup_latitude'].values, df['pickup_longitude'].values, df['dropoff_latitude'].values, df['dropoff_longitude'].values)
    #JFK Distance     
        df.loc[:,'distance_jfk_pick'] = haversine_array(df['pickup_latitude'].values, df['pickup_longitude'].values, df['jfk_lat'].values, df['jfk_lon'].values)
        df.loc[:,'distance_jfk_drop'] = haversine_array(df['dropoff_latitude'].values, df['dropoff_longitude'].values, df['jfk_lat'].values, df['jfk_lon'].values)
    #Airport Distance    
        df.loc[:,'distance_laguardia_pick'] = haversine_array(df['pickup_latitude'].values, df['pickup_longitude'].values, df['la_lat'].values, df['la_lon'].values)
        df.loc[:,'distance_laguardia_drop'] = haversine_array(df['dropoff_latitude'].values, df['dropoff_longitude'].values, df['la_lat'].values, df['la_lon'].values)
        
        df.drop(['jfk_lat', 'jfk_lon', 'la_lat', 'la_lon'], axis = 1, inplace = True)
    return train, test

1. Outlier
https://www.kaggle.io/svf/1512124/63b262954485433518d65fb2927eb518/__results__.html#direct-distance-of-the-trip

In [8]:
def cleanDf(train):
    train = train.loc[(train['trip_duration'] < 22*3600)]
    train = train.loc[(train['distance_haversine'] > 0) | ((train['distance_haversine'] == 0) & (train['trip_duration'] < 60))]
    train = train.loc[(train['distance_jfk_pick'] < 3e5 ) & (train['distance_jfk_drop']<3e5)]
    train = train.loc[(train['trip_duration'] > 10)]
    return train

In [14]:
def mergeRouteWeather(train, test):
    
    #Fast Data
    fast1 = pd.read_csv('../NYC/AdditionalSource/fastest_routes_train_part_1.csv', usecols = ['id','total_distance', 'total_travel_time', 'number_of_steps'])
    fast2 = pd.read_csv('../NYC/AdditionalSource/fastest_routes_train_part_2.csv', usecols = ['id','total_distance', 'total_travel_time', 'number_of_steps'])
    fast = pd.concat([fast1, fast2], axis = 0)
    #Weather Data
    weather = pd.read_csv('../NYC/AdditionalSource/KNYC.csv', usecols = ['Time','Conditions'])
    weather['Time'] = pd.to_datetime(weather['Time'])
    weather['yday'] = weather['Time'].dt.dayofyear
    weather['hour'] = weather['Time'].dt.hour
    weather.drop(['Time'], axis =1, inplace = True)
    weather.rename(columns = {'Conditions' : 'condition'}, inplace = True)
    weather.drop_duplicates(subset = ['yday', 'hour'], keep = 'first', inplace = True)
    
    #Merging Fast And Weather
    ix = 0
    for df in [train, test]:
        df = pd.merge(df, fast, on = ['id'], how = 'left')
        df = pd.merge(df, weather, on = ['yday', 'hour'], how = 'left')
        if ix == 0: train = df
        else: test = df
        ix += 1
    
    return train, test
        
        

Get dummies by Pandas, adding prefix and seperation
https://www.kaggle.com/karelrv/nyct-from-a-to-z-with-xgboost-tutorial

In [12]:
def dummyBuilding(train, test):
    ix = 0
    for df in [train, test]:
        df_pickup_cluster = pd.get_dummies(df['pickup_cluster'], prefix = 'p', prefix_sep = '_')
        df_dropoff_cluster = pd.get_dummies(df['dropoff_cluster'], prefix = 'd', prefix_sep = '_')
        #df_month = pd.get_dummies(df['month'], prefix = 'm', prefix_sep = '_')
        df_wday = pd.get_dummies(df['wday'], prefix = 'd', prefix_sep ='_')
        df_hour = pd.get_dummies(df['hour'], prefix = 'h', prefix_sep = '_')
        #df_condition = pd.get_dummies(df['condition'], prefix = 'c', prefix_sep = '_')
        
        #if ix == 1: df_passenger_count = df_passenger_count.drop('pc_9', axis = 1)
        #check condition regard of test and train
        
        df.drop(['passenger_count','pickup_datetime'], axis = 1, inplace = True)
        df.drop(['wday', 'hour', 'condition'], axis = 1, inplace = True)
        if ix == 0: df.drop(['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'], axis =1, inplace = True)
        else: df.drop(['pickup_longitude','pickup_latitude'], axis =1, inplace = True)
        
        df_Master = pd.concat([df, df_pickup_cluster, df_dropoff_cluster, 
                              df_wday, df_hour, df_condition], axis = 1)
        del df, df_pickup_cluster, df_dropoff_cluster, df_wday, df_hour, df_condition
        
        if ix == 0: train = df_Master
        else: test = df_Master
        ix += 1
    
    return train, test    
    
        